In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install timm
# !pip install imageio-ffmpeg
# !pip install segmentation-models-pytorch
# !pip install colorama
# !pip install einops
# !pip install statsmodels
# !pip install transformers
# !pip install kaggle
# !pip install --upgrade https://github.com/VincentStimper/mclahe/archive/numpy.zip -q
# # !pip install git+https://github.com/asullivan42/mclahe.git
# # # # # # !pip install torchsummary
# # # # # !pip install csbdeep

In [ ]:
# !cp /content/drive/MyDrive/kaggle_competitions/vision/INK_segmentation_2023/input/resnet3d.py /content/

In [ ]:
# utils
import os
import sys
import random
import tifffile
import gc
from glob import glob
from datetime import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import cv2
import PIL.Image as Image
Image.MAX_IMAGE_PIXELS = 10000000000
import collections
from einops import rearrange, reduce, repeat

# augmentation
import albumentations as A
from albumentations.pytorch import ToTensorV2

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda import amp
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
# import torchsummary

# scipy stat
import scipy
from scipy.stats import median_abs_deviation


#model
sys.path.append('/content/drive/MyDrive/kaggle_competitions/vision/INK_segmentation_2023/input')
from resnet3d import generate_model
import segmentation_models_pytorch as smp
from timm.models.resnet import *
import timm
from segmentation_models_pytorch.decoders.unet.decoder import UnetDecoder, DecoderBlock

# kfold
from sklearn.model_selection import KFold

# gauss filter - median
from scipy.ndimage import gaussian_filter


# 3d-clahe
import mclahe as mc
import tensorflow

# color font
from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

from typing import Dict
from google.colab import _debugpy_repr
import scipy.io

class dotdict(dict):
    def __getattr__(self, item):
        try:
            return self[item]
        except KeyError as e:
            raise AttributeError from e

    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

a=1
b=1
d = dotdict(a=b)
_debugpy_repr.get_shape(d)

In [ ]:
# !mkdir /root/.kaggle
# !cp /content/drive/MyDrive/kaggle_competitions/kaggle.json /root/.kaggle  # kaggl.json위치 지정
# !chmod 600 /root/.kaggle/kaggle.json
# !kaggle competitions download -c vesuvius-challenge-ink-detection

# !unzip -qq  /content/vesuvius-challenge-ink-detection.zip

In [ ]:
class CFG(object):

    # 2 -> 14830, 9506,

    crop_size = 224
    crop_fade = crop_size//4
    crop_stride = crop_size//2
    num_concat = 2
    crop_depth = 5
    train_fragment_z = [18, 42] # 20 range ->  16+17, 17+18
    infer_fragment_z = [18, 42]
    train_fragment_depth = train_fragment_z[1] - train_fragment_z[0]
    infer_fragment_depth = infer_fragment_z[1] - infer_fragment_z[0]

    is_tta = True

    train_batch_size = 32
    valid_batch_size = 32
    max_grad_norm = 1000
    accumulate_iter = 2


    lr = 1e-3
    wd = 1e-4
    eps = 1e-6
    num_epochs = 50
    patience = 6
    num_workers = 1
    batch_scheduler = True
    seed = 2023
    date = datetime.now().strftime("%Y%m%d")
    model_name = "colabv7_resnet34d_unet_stride"
    # model_name = "colabv4_seresnext26t_32x4d_unet"
    train_model_name = 'resnet34d'
    # train_model_name = 'seresnext26t_32x4d'
    # "seresnext26t_32x4d"

    train_fragment = np.array(['1a',"1b","3","2e","2f"])
    valid_fragment = "1"
    n_splits = len(train_fragment)

    colab_dir = '/content/drive/MyDrive/kaggle_competitions/vision/INK_segmentation_2023'

    train_dir = '/content/train'
    mask_dir = colab_dir + "/input"
    private_dir =  colab_dir + "/input"

    mode = [
        'train', #
#         'test', 'skip_fake_test',
    ]

    device =  "cuda" if torch.cuda.is_available() else "cpu"
os.environ['CUDA_VISIBLE_DEVICES'] ='0'
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"



cfg = CFG()

if 'train' in cfg.mode:
    data_dir = '/content/train'
    valid_id =[
        '1',#'2b',
    ]


print('data_dir', data_dir)
print('valid_id', valid_id)

data_dir /content/train
valid_id ['1']


In [ ]:
def pixel_clipping(img, low_b, high_b):
    img = np.array(img)
    display(f"img mean : {img.mean()}, img mean : {img.max()}, img mean : {img.min()}")

    lo_pct, hi_pct = 5.0, 95.0
    lo_val, hi_val = np.percentile(img, [lo_pct, hi_pct])
    new_img = np.clip(img, lo_val, hi_val)
    new_img = ((new_img - new_img.min()) * 255.0 / (new_img.max() - new_img.min())).astype(np.uint8)
    display(f"new img mean : {new_img.mean()}, new img mean : {new_img.max()}, new img mean : {new_img.min()}")

    return new_img

def median_state(volume_img, volume_mask):
    eps = 1e-7
    h,w,c = volume_img.shape
    volume_img = volume_img.transpose(2,0,1) # [c, h, w]
    median = np.median(volume_img[:,volume_mask.astype(bool)], axis=1, overwrite_input=False)
    mad = np.median(
        np.abs(volume_img[:,volume_mask.astype(bool)].T - median),
         axis=0, overwrite_input=True)

    return median, mad

def median_normalize(volume_img, volume_mask):
    """
    img = [h,w,c]
    """
    # median_scale
    eps = 1e-8
    h,w,c = volume_img.shape
    volume_img = volume_img.transpose(2,0,1) # [c, h, w]
    median = np.median(volume_img[:,volume_mask.astype(bool)], axis=1, overwrite_input=False).astype(np.float16)
    mad = np.median(
        np.abs(volume_img[:,volume_mask.astype(bool)].T - median),
         axis=0, overwrite_input=True).astype(np.float16)
    imgs = np.zeros((h,w,c), dtype=np.float16)
    for i in tqdm(range(c)):
        imgs[...,i] = ((volume_img[i] - median[i]) / (mad[i] + eps))
#     imgs = imgs.astype(np.uint8)
    # imgs = np.stack(imgs, axis=2) # [h,w,c]

    # minmax
    # max = imgs.max()
    # min = imgs.min()
    # imgs = ((imgs - min) / (max - min))
    return imgs

# Ordering

1. 모든 layer load
2. cliping
3. 원하는 layer 선택
4. train


In [ ]:
image_map = {
    '1': {'h': 8181, 'w': 6330},
    '2': {'h': 14830, 'w': 9506},
    '3': {'h': 7606, 'w': 5249}
    }

In [ ]:
def do_3dcleha(images):
        """
        image shape :  c, h, w
        tensor
        """

        images  = mc.mclahe(images, kernel_size=[5,8,8],
                n_bins=64,
                clip_limit=0.01,
                adaptive_hist_range=False,
                )

        return images

def timer():
    now = time.time()
    return now

def do_binary(mask, thr=0.5):
    mask = mask-mask.min()
    mask = mask/(mask.max()+1e-7)
    mask = (mask>thr).astype(np.float32) # np.float32

    return mask

def read_data(fragment_id, z0=cfg.train_fragment_z[0], z1=cfg.train_fragment_z[1]):
    start_timer = timer()

    # full load
    images = []
    for i in range(z0, z1):
        image = cv2.imread(f"{data_dir}/{fragment_id}/surface_volume/{i:02d}.tif",0)
        # image = (image>>8).astype(np.uint8)
        images.append(image)

        print(f'\r @ read_data(): images{fragment_id}  {f"{timer() - start_timer}"+ "sec"}', end='', flush=True)

    images = np.stack(images, 2)

    mask = cv2.imread(f"{cfg.mask_dir}/{fragment_id}_adjusted_mask.png", cv2.IMREAD_GRAYSCALE)
    images[mask == 0] = 0
    mask = do_binary(mask)

    # del images_

    if 'train' in cfg.mode:
#         ir = cv2.imread(f'{data_dir}/{fragment_id}/ir.png', cv2.IMREAD_GRAYSCALE)
        # label = cv2.imread(f'{data_dir}/{fragment_id}/inklabels.png', cv2.IMREAD_GRAYSCALE)
        label = cv2.imread(f"{cfg.private_dir}/new_inklabel{fragment_id}.png",cv2.IMREAD_GRAYSCALE)
#         ir = ir / 255.0
        label = do_binary(label)

    if 'test' in cfg.mode:
        ir = None
        label = None

    dic = dotdict(
        fragment_id = fragment_id,
        images = images,
#         ir = ir,
        label = label,
        mask = mask
    )


    gc.collect()
    return dic

def read_dic(fragment_id):
    if fragment_id == "2a":
        y = 7050
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2a',
            images = dic.images[:y],
#             ir = dic.ir[:y],
            label = dic.label[:y],
            mask = dic.mask[:y]
        )
    elif fragment_id=="2b":
        y0,y1 = 7050, 10500
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2b',
            images = dic.images[y0:y1],
#             ir = dic.ir[y:],
            label = dic.label[y0:y1],
            mask = dic.mask[y0:y1]
        )
    elif fragment_id=="2c":
        y = 10500
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2c',
            images = dic.images[y:],
#             ir = dic.ir[y:],
            label = dic.label[y:],
            mask = dic.mask[y:]
        )
    elif fragment_id=="2d":
        y0,y1 = 0, 7074
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2d',
            images = dic.images[y0:y1],
#             ir = dic.ir[y:],
            label = dic.label[y0:y1],
            mask = dic.mask[y0:y1]
        )

    elif fragment_id=="2e":
        y = 9456
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2d',
            images = dic.images[:y],
#             ir = dic.ir[y:],
            label = dic.label[:y],
            mask = dic.mask[:y]
        )
    elif fragment_id=="2f":
        y = 9456
        dic = read_data('2')
        dic = dotdict(
            fragment_id = '2f',
            images = dic.images[y:],
#             ir = dic.ir[y:],
            label = dic.label[y:],
            mask = dic.mask[y:]
        )

    elif fragment_id=="1a":
        y = 2600
        dic = read_data('1')
        dic = dotdict(
            fragment_id = '1a',
            images = dic.images[:y],
#             ir = dic.ir[y:],
            label = dic.label[:y],
            mask = dic.mask[:y]
        )

    elif fragment_id=="1b":
        y = 2600
        dic = read_data('1')
        dic = dotdict(
            fragment_id = '1b',
            images = dic.images[y:],
#             ir = dic.ir[y:],
            label = dic.label[y:],
            mask = dic.mask[y:]
        )


    elif fragment_id=="3":
#         y = 9456
        dic = read_data('3')
        dic = dotdict(
            fragment_id = '3',
            images = dic.images,
#             ir = dic.ir[y:],
            label = dic.label,
            mask = dic.mask
        )

    else:
        dic = read_data(fragment_id)

    gc.collect()
    return dic

In [ ]:
data = read_dic("1")

 @ read_data(): images1  24.301363229751587sec

In [ ]:
def run_check_data():
    dic=read_dic(valid_id[0])#valid_id[0]
    display('')
    display('fragment_id:', dic.fragment_id)
    display('images:', dic.images.shape, dic.images.min(), dic.images.max())
#     display('mask  :', dic.mask.shape, dic.mask.min(), dic.mask.max())
    if 'train' in cfg.mode:
        display('ir    :', dic.ir.shape, dic.ir.min(), dic.ir.max())
        display('label :', dic.label.shape, dic.label.min(), dic.label.max())

# run_check_data()
print('data ok !!!')



data ok !!!


In [ ]:
def get_train_transforms():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.OneOf([
            A.Sharpen(alpha=0.1, lightness=0.1, p=0.3),
            # A.RandomGamma(gamma_limit=(50,100), p=0.3),
            A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15,p=0.3),
        ],p=0.4),
        A.ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=0,
            p=0.5),
        A.Transpose(p=0.2),
        A.OneOf([
                A.GaussNoise(var_limit=0.08, p=0.5), # var_limit=[2, 10]
                A.GaussianBlur(p=0.2),
                A.MotionBlur(p=0.2),
                # A.ISONoise(color_shift=(0.01,0.03), intensity=(0.1,0.2)),
                A.MedianBlur(blur_limit=5, p=0.3),
                A.MultiplicativeNoise(multiplier=(0.9,1.1), p=0.1),
                ], p=0.5),

        A.OneOf([
                A.OpticalDistortion(p=0.2),
                A.GridDistortion(num_steps=5, distort_limit=0.2, p=0.5),
                A.Affine(p=0.2),
                A.RandomGridShuffle(grid=(2,2), p=0.2),
                A.PiecewiseAffine(p=0.1),
        ], p=0.5),

        A.OneOf([
#             A.Cutout(p=0.3, num_holes=4, max_h_size=int(CFG.crop_size * 0.2), max_w_size=int(CFG.crop_size * 0.2)),
            A.CoarseDropout(max_holes=2, max_width=int(cfg.crop_size * 0.2), max_height=int(cfg.crop_size * 0.2),
                        mask_fill_value=0, p=0.3),
            A.GridDropout(ratio=0.1, random_offset=True, mask_fill_value=0, p=0.3),
        ], p=0.4),

        A.Normalize(mean=[0]*cfg.train_fragment_depth, std=[1]*cfg.train_fragment_depth),
        ToTensorV2(transpose_mask=True)
    ], additional_targets={'mask1': 'mask'})

def get_valid_transforms():
    return A.Compose([
        A.Normalize(mean=[0]*cfg.train_fragment_depth, std=[1]*cfg.train_fragment_depth),
        ToTensorV2(transpose_mask=True)
    ],  additional_targets={'mask1': 'mask'})

In [ ]:
class CustomTrainDataset(Dataset):
    def __init__(self, fragment_id, transforms = None):
        self.fragment_id = fragment_id
        self.transforms = transforms
        self.norm_transform = A.Compose([
            A.Normalize(mean=[0]*cfg.train_fragment_depth, std=[1]*cfg.train_fragment_depth),
            ToTensorV2(transpose_mask=True)
        ],p=1, additional_targets={"mask1" : "mask"})

        self.xys = []
        self.xyss = None
        self.dics = {}

        for fragment in fragment_id:

            self.dics[fragment] = read_dic(fragment)
            H,W,C = self.dics[fragment]['images'].shape
            imgs = []
            for i in range(C):
                pad0 = (cfg.crop_stride - H % cfg.crop_stride)
                pad1 = (cfg.crop_stride - W % cfg.crop_stride)
                img = np.pad(self.dics[fragment]['images'][:,:,i], [(0, pad0), (0, pad1)], constant_values=0)
                imgs.append(img)
            imgs = np.stack(imgs, axis=2)
            self.dics[fragment]['images'] = imgs
            H,W,C = self.dics[fragment]['images'].shape

            for y in range(0, H-cfg.crop_size+1, cfg.crop_size):
                for x in range(0, W-cfg.crop_size+1, cfg.crop_size):
                    self.xys.append((fragment, x, y, W, H))

            self.dics[fragment]['label'] = np.pad(self.dics[fragment]['label'], [(0, pad0), (0, pad1)], constant_values=0)
            self.dics[fragment]['mask'] = np.pad(self.dics[fragment]['mask'], [(0, pad0), (0, pad1)], constant_values=0)

        self.xyss = self.filter_empty_label(self.xys)

    def __len__(self):
        return len(self.xyss)


    def __getitem__(self, idx):

        fragment, x1, y1, W, H = self.xyss[idx]


        # augment
        if np.random.rand()<0.7:
            if np.random.rand()<0.7:
                frag_crop, label_crop, mask_crop, crop_x1,crop_y1 = self.do_random_crop(
                self.dics[fragment]['images'],
                self.dics[fragment]['label'],
                self.dics[fragment]['mask'],
                crop_size=cfg.crop_size,
                valid_mask_threshold=0.8)

                x2 = crop_x1 + cfg.crop_size
                y2 = crop_y1 + cfg.crop_size


            else:

                x2 = x1 + cfg.crop_size
                y2 = y1 + cfg.crop_size

                frag_crop = self.dics[fragment]['images'][y1:y2, x1:x2, :]
                label_crop = self.dics[fragment]['label'][y1:y2, x1:x2]
                mask_crop = self.dics[fragment]['mask'][y1:y2, x1:x2]

            frag_crop = np.ascontiguousarray(frag_crop)
            label_crop = np.ascontiguousarray(label_crop)
            mask_crop = np.ascontiguousarray(mask_crop)


            if self.transforms is not None:
                data = self.transforms(image=frag_crop, mask=label_crop, mask1=mask_crop)
                images = data['image']
                labels = data['mask']
                masks = data['mask1']

        else:

            if np.random.rand()<0.7:
                # crop_types = ['mask','label']
                # crop_type = np.random.choice(crop_types)
                frag_crop, label_crop, mask_crop, crop_x1,crop_y1 = self.do_random_crop(
                self.dics[fragment]['images'],
                self.dics[fragment]['label'],
                self.dics[fragment]['mask'],
                crop_size=cfg.crop_size,
                valid_mask_threshold=0.8)

                x2 = crop_x1 + cfg.crop_size
                y2 = crop_y1 + cfg.crop_size

            else:
                x2 = x1 + cfg.crop_size
                y2 = y1 + cfg.crop_size
                frag_crop = self.dics[fragment]['images'][y1:y2, x1:x2, :]
                label_crop = self.dics[fragment]['label'][y1:y2, x1:x2]
                mask_crop = self.dics[fragment]['mask'][y1:y2, x1:x2]

            if np.random.rand()<0.5:
                frag_crop = cv2.flip(frag_crop, 1)
                label_crop = cv2.flip(label_crop, 1)
                mask_crop = cv2.flip(mask_crop, 1)

            if np.random.rand()<0.5:
                frag_crop = cv2.flip(frag_crop, 0)
                label_crop = cv2.flip(label_crop, 0)
                mask_crop = cv2.flip(mask_crop, 0)

            if np.random.rand()<0.4:
                n = np.random.choice(4)
                frag_crop = np.rot90(frag_crop, n, axes=(0,1))
                label_crop = np.rot90(label_crop, n, axes=(0,1))
                mask_crop = np.rot90(mask_crop, n, axes=(0,1))

            if np.random.rand()<0.4:
                type_list = ["gauss", "poisson","speckle"]
                noise_type = np.random.choice(type_list)
                frag_crop = self.do_random_noise(frag_crop, noise_type)

            frag_crop = np.ascontiguousarray(frag_crop)
            label_crop = np.ascontiguousarray(label_crop)
            mask_crop = np.ascontiguousarray(mask_crop)

            data = self.norm_transform(image=frag_crop, mask=label_crop, mask1=mask_crop)
            images = data['image']
            labels = data['mask']
            masks = data['mask1']

        return {"images":  images ,# .type(torch.FloatTensor)
                "labels" : labels,
                "masks" : masks,
                "xys" : torch.tensor([x1, y1, x2, y2], dtype=torch.int32)
               }


    def filter_empty_label(self,xyss):
        new_xys = []
        for xys in tqdm(xyss):
            fragment, x1, y1, W, H = xys
            x = x1
            y = y1

            x2 = x1 + cfg.crop_size
            y2 = y1 + cfg.crop_size

            mask_crop = self.dics[fragment]['mask'][y1:y2, x1:x2]
            if mask_crop.mean()>0.05:
                new_xys.append((fragment, x, y, W, H))

        return new_xys

    def do_random_crop(
        self,
        image,
        label,
        mask,
        crop_size=cfg.crop_size,
        valid_mask_threshold=0.8
    ):
        height, width = image.shape[:2]
        while (1):
            y = np.random.randint(0, height - crop_size+1)
            x = np.random.randint(0, width  - crop_size+1)
            crop_image = image[y:y + crop_size, x:x + crop_size,:]
            crop_mask  = mask[y:y + crop_size, x:x + crop_size]
            crop_label  = label[y:y + crop_size, x:x + crop_size]

            if crop_mask.mean() > valid_mask_threshold:
                break

        return crop_image, crop_label, crop_mask, x, y


    def do_equalize(self, images):
        """
        input shape : [h,w,c]
        """
        images = images.transpose(2,0,1)
        images = np.stack([cv2.equalizeHist(image) for image in images],-1)

        return images

    def do_clahe(self, images):
        """
        input shape  : [h,w,c]
        """
        images = images.transpose(2,0,1) # 224, 224
        limit_list = [2, 4]
        grid_list = [2,4,8]
        limit = np.random.choice(limit_list,1)
        grid = np.random.choice(grid_list,1)
        clahe  = cv2.createCLAHE(clipLimit=limit[0], tileGridSize=(grid[0],grid[0]))
        images = np.stack([clahe.apply(image) for image in images], -1)

        return images

    def do_random_noise(self, images, noise_type):
        if noise_type == "gauss":
            row,col,ch= images.shape
            mean = 0
            var = 0.1
            sigma = var**0.5
            gauss = np.random.normal(mean,sigma,(row,col,ch))
            gauss = gauss.reshape(row,col,ch)
            noisy = images + gauss
            return noisy
        elif noise_type == "poisson":
            vals = len(np.unique(images))
            vals = 2 ** np.ceil(np.log2(vals))
            noisy = np.random.poisson(images * vals) / float(vals)
            return noisy
        elif noise_type =="speckle":
            row,col,ch = images.shape
            gauss = np.random.randn(row,col,ch)
            gauss = gauss.reshape(row,col,ch)
            noisy = images + images * gauss
            return noisy

    def do_3dcleha(self, images):
        """
        image shape : h,w,c or c, h, w
        tensor
        """

        images = images.permute(2,0,1) # 224, 224
        images = images.numpy()
        images  = mc.mclahe(images, kernel_size=[4,4,4],
                n_bins=64,
                clip_limit=0.01,
                adaptive_hist_range=False,

                )
        images = torch.tensor(images, dtype=torch.float32)
        images = images.permute(1,2,0)
        return images


class CustomValidDataset(Dataset):
    def __init__(self, fragment_id, transforms = None):
        self.fragment_id = fragment_id
        self.transforms = transforms
        self.xys = []
        self.dics = {}

        for fragment in fragment_id:
            self.dics[fragment] = read_dic(fragment)
            H,W,C = self.dics[fragment]['images'].shape
            imgs = []
            for i in range(C):
                pad0 = (cfg.crop_stride - H % cfg.crop_stride)
                pad1 = (cfg.crop_stride - W % cfg.crop_stride)
                img = np.pad(self.dics[fragment]['images'][:,:,i], [(0, pad0), (0, pad1)], constant_values=0)
                imgs.append(img)
            imgs = np.stack(imgs, axis=2)
            self.dics[fragment]['images'] = imgs

            H,W,C = self.dics[fragment]['images'].shape

            for y in range(0, H-cfg.crop_size+1, cfg.crop_stride):
                for x in range(0, W-cfg.crop_size+1, cfg.crop_stride):
                    self.xys.append((fragment, x, y, W, H))

            self.dics[fragment]['label'] = np.pad(self.dics[fragment]['label'], [(0, pad0), (0, pad1)], constant_values=0)
            self.dics[fragment]['mask'] = np.pad(self.dics[fragment]['mask'], [(0, pad0), (0, pad1)], constant_values=0)

    def __len__(self):
        return len(self.xys)

    def __getitem__(self, idx):
        fragment, x1, y1, W, H = self.xys[idx]

        x2 = x1 + cfg.crop_size
        y2 = y1 + cfg.crop_size

        frag_crop = self.dics[fragment]['images'][y1:y2, x1:x2, :]
        label_crop = self.dics[fragment]['label'][y1:y2, x1:x2]
        mask_crop = self.dics[fragment]['mask'][y1:y2, x1:x2]

        # augment
        if self.transforms is not None:
            data = self.transforms(image=frag_crop, mask=label_crop, mask1=mask_crop)
            images = data['image']
            labels = data['mask']
            masks = data['mask1']

        return {"images": images.type(torch.FloatTensor) , # .type(torch.FloatTensor)
                "labels" : labels,
                "masks" : masks,
                "xys" : torch.tensor([x1, y1, x2, y2], dtype=torch.int32)
               }

    def do_equalize(self, images):
        """
        input shape : [h,w,c]
        """
        images = images.transpose(2,0,1)
        images = np.stack([cv2.equalizeHist(image) for image in images],-1)

        return images

    def do_clahe(self, images):
        """
        input shape  : [h,w,c]
        """
        limit_list = [2, 4]
        grid_list = [2,4,8]
        limit = np.random.choice(limit_list,1)
        grid = np.random.choice(grid_list,1)
        clahe  = cv2.createCLAHE(clipLimit=limit[0], tileGridSize=(grid[0],grid[0]))
        images = np.stack([clahe.apply(image) for image in images], -1)

        return images

    def do_3dcleha(self, images):
            """
            tensor
            image shape : h,w,c or c, h, w
            """
            images = images.permute(2,0,1) # 224, 224
            images = images.numpy()
            images  = mc.mclahe(images, kernel_size=[4,4,4],
                    n_bins=64,
                    clip_limit=0.01,
                    adaptive_hist_range=False,
                    )

            images = torch.tensor(images, dtype=torch.float32)
            images = images.permute(1,2,0)
            return images

def collate_fn(batch):
    return {
        'images': torch.stack([x['images'] for x in batch]),
        'labels': torch.stack([x['labels'] for x in batch]),
        'masks': torch.stack([x['masks'] for x in batch]),
        'xys'   : torch.stack([x['xys'] for x in batch])
}

In [ ]:
# dataloader test
# train_dataset = CustomTrainDataset("1", get_train_transforms())
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
# data = next(iter(train_loader))


In [ ]:
# for i, data in enumerate(train_loader):
#     print(f"{i+1} / {len(train_loader)}")
#     print(len(data))
#     for j in range(len(data['images'])):
#         plt.figure(figsize=(12,8))
#         plt.subplot(1,3,1)
#         plt.imshow(data['images'][j][0])
#         plt.subplot(1,3,2)
#         plt.imshow(data['labels'][j], vmin=0, vmax=1)
#         plt.subplot(1,3, 3)
#         plt.imshow(data['masks'][j], vmin=0, vmax=1)
#         plt.title(f"mask size : {data['masks'][j].mean()}")
#         plt.show()

# Model

In [ ]:
# model
class SmpUnetDecoder(nn.Module):
    def __init__(self,
             in_channel,
             skip_channel,
             out_channel,
        ):
        super().__init__()
        self.center = nn.Identity()

        i_channel = [in_channel,]+ out_channel[:-1]
        s_channel = skip_channel
        o_channel = out_channel
        block = [
            DecoderBlock(i, s, o, use_batchnorm=True, attention_type=None)
            for i, s, o in zip(i_channel, s_channel, o_channel)
        ]
        self.block = nn.ModuleList(block)

    def forward(self, feature, skip):
        d = self.center(feature)
        decode = []
        for i, block in enumerate(self.block):
            s = skip[i]
            d = block(d, s)
            decode.append(d)

        last  = d
        return last, decode


class Net(nn.Module):
    def __init__(self, ):
        super().__init__()

        # --------------------------------
        self.crop_depth = cfg.crop_depth

        conv_dim = 64
        encoder_dim = [conv_dim, 64, 128, 256, 512]
        decoder_dim = [256, 128, 64, 32, 16 ]
        self.encoder = resnet34d(pretrained=True, in_chans=self.crop_depth, drop_rate=0.5)

        self.decoder = SmpUnetDecoder(
            in_channel=encoder_dim[-1],
            skip_channel=encoder_dim[:-1][::-1] + [0],
            out_channel=decoder_dim,
        )
        self.logit = nn.Conv2d(decoder_dim[-1], 1, kernel_size=1)

        # aux
        self.aux = nn.ModuleList([
            nn.Conv2d(encoder_dim[i], 1, kernel_size=1, padding=0) for i in range(len(encoder_dim))
        ])

        # attentin pool
        self.weight = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(dim, dim, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
            ) for dim in encoder_dim
        ])

        # dropout layer
        self.dropout1 = nn.Dropout(0.5)

    def forward(self, batch):
        v = batch['images']
        B, C, H, W = v.shape
        vv = [
            v[:, i:i + self.crop_depth] for i in range(0,C-self.crop_depth+1,2)
        ]
        K = len(vv)
        x = torch.cat(vv, 0)

        # ---------------------------------

        encoder = []

        e = self.encoder

        x = e.conv1(x)
        x = e.bn1(x)
        x = e.act1(x)
        encoder.append(x)
        x = F.avg_pool2d(x, kernel_size=2, stride=2)
        x = e.layer1(x); encoder.append(x);
        x = e.layer2(x); encoder.append(x);
        x = e.layer3(x); encoder.append(x);
        x = e.layer4(x); encoder.append(x);

        # torch.Size([14, 64, 112, 112]) x
        # torch.Size([14, 64, 56, 56]) x pool
        # torch.Size([14, 256, 56, 56])x1
        # torch.Size([14, 512, 28, 28])x2
        # torch.Size([14, 1024, 14, 14])x3
        # torch.Size([14, 2048, 7, 7]) x 4
        # torch.Size([14, 64, 112, 112]) e
        # torch.Size([14, 256, 56, 56]) e

        for i in range(len(encoder)):
            e = encoder[i]
            _, c, h, w = e.shape # k b 512, 7, 7
            e = rearrange(e, '(K B) c h w -> K B c h w', K=K, B=B, h=h, w=w)

            encoder[i] = e.mean(0)


        last, decoder = self.decoder(feature = encoder[-1], skip = encoder[:-1][::-1]  + [None])


        # ---------------------------------
        logit = self.logit(last)

        output = {}
        output['logit'] = logit

        if 1:
            if logit.shape[2:]!=(H, W):
                logit = F.interpolate(logit, size=(H, W), mode='bilinear', align_corners=False, antialias=True)
            output['ink'] = torch.sigmoid(logit)

        return output

In [ ]:
# model I/O test

# batch = {
#         'images' : torch.from_numpy( np.random.choice(256, (1, cfg.train_fragment_depth, 224, 224))).float(),
#     }

# model = Net()
# model(batch)['logit'].shape

# Metric and Losses

In [ ]:
class CompoundLoss(nn.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # self.diceloss= smp.losses.DiceLoss(mode='binary', from_logits=True)
        self.bceloss = smp.losses.SoftBCEWithLogitsLoss()
        # self.bceloss = torch.nn.BCEWithLogitsLoss()
        self.bceloss.__name__ = 'bce_loss'

    def forward(self, y_pr, y_gt):
        return self.bceloss.forward(y_pr,y_gt)
        # return self.diceloss.forward(y_pr, y_gt)
        # return (self.bceloss.forward(y_pr, y_gt) + self.diceloss.forward(y_pr, y_gt)) / 2
        # return 0.7 * self.diceloss.forward(y_pr, y_gt) + 0.3 * self.bceloss.forward(y_pr, y_gt)

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        # inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy_with_logits(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha, beta, gamma, smooth, weight=None, size_average=True):
        super(FocalTverskyLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        #True Positives, False Positives & False Negatives
        TP = (inputs * targets).sum()
        FP = ((1-targets) * inputs).sum()
        FN = (targets * (1-inputs)).sum()

        Tversky = (TP + self.smooth) / (TP + self.alpha*FP + self.beta*FN + smooth)
        FocalTversky = (1 - Tversky)**self.gamma

        return FocalTversky

def fbeta_score(preds, targets, threshold, beta=0.5, smooth=1e-5):
    preds_t = torch.where(preds > threshold, 1.0, 0.0).float()
    y_true_count = targets.sum()

    ctp = preds_t[targets==1].sum()
    cfp = preds_t[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)
    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

def fbeta_numpy(targets, preds, beta=0.5, smooth=1e-5):
    """
    https://www.kaggle.com/competitions/vesuvius-challenge-ink-detection/discussion/397288
    """
    y_true_count = targets.sum()
    ctp = preds[targets==1].sum()
    cfp = preds[targets==0].sum()
    beta_squared = beta * beta

    c_precision = ctp / (ctp + cfp + smooth)

    c_recall = ctp / (y_true_count + smooth)
    dice = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + smooth)

    return dice

def make_infer_mask():
    s = cfg.crop_size
    f = cfg.crop_fade
    x = np.linspace(-1, 1, s)
    y = np.linspace(-1, 1, s)
    xx, yy = np.meshgrid(x, y)
    d = 1 - np.maximum(np.abs(xx), np.abs(yy))
    d1 = np.clip(d, 0, f / s * 2)
    d1 = d1 / d1.max()
    infer_mask = d1
    return torch.from_numpy(infer_mask).float().cuda()

def mask_to_rle(mask):
    m = mask.reshape(-1)

    s = np.array((m[:-1] == 0) & (m[1:] == 1))
    e = np.array((m[:-1] == 1) & (m[1:] == 0))

    s_index = np.where(s)[0] + 2
    e_index = np.where(e)[0] + 2
    length = e_index - s_index
    rle = ' '.join(map(str, sum(zip(s_index, length), ())))
    return rle

def metric_to_text(ink, label, mask):
    text = []
    p = ink.reshape(-1)
    t = label.reshape(-1)
    pos = np.log(np.clip(p,1e-7,1))
    neg = np.log(np.clip(1-p,1e-7,1))
    bce = -(t*pos +(1-t)*neg).mean()
    text.append(f'bce={bce:0.5f}')


    mask_sum = mask.sum()

    text.append('p_sum  th   prec   recall   fpr   dice   score')
    text.append('-----------------------------------------------')
    for threshold in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        p = ink.reshape(-1)
        t = label.reshape(-1)
        p = (p > threshold).astype(np.float32)
        t = (t > 0.5).astype(np.float32)

        tp = p * t
        precision = tp.sum() / (p.sum() + 0.0001)
        recall = tp.sum() / t.sum()

        fp = p * (1 - t)
        fpr = fp.sum() / (1 - t).sum()

        beta = 0.5
        score = beta * beta / (1 + beta * beta) * 1 / recall + 1 / (1 + beta * beta) * 1 / precision
        score = 1 / score

        dice = 2 * tp.sum() / (p.sum() + t.sum())
        p_sum = p.sum()/mask_sum

        text.append( f'{p_sum:0.2f}, {threshold:0.2f}, {precision:0.3f}, {recall:0.3f}, {fpr:0.3f},  {dice:0.3f},  {score:0.3f}')

    del p, t, tp, precision, recall, fp, fpr, score, dice, p_sum
    gc.collect()
    text = '\n'.join(text)
    return text


In [ ]:
class EarlyStopping():
    def __init__(self, patience=0, verbose=0):
        self._step = 0
        self._score = 0
        self._loss = float('inf')
        self.patience  = patience
        self.verbose = verbose

    def loss_validate(self, loss):
        if self._loss < loss:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print(f'Training process is stopped early....')
                return True
        else:
            self._step = 0
            self._loss = loss

    def score_validate(self,score):
        if self._score > score:
            self._step += 1
            if self._step > self.patience:
                if self.verbose:
                    print(f"Training process is stopped early....")
                return True
        else:
            self._step = 0
            self._score = score

In [ ]:
kf = KFold(n_splits=cfg.n_splits, shuffle=False)
for fold, (trn_idx, val_idx) in enumerate(kf.split(cfg.train_fragment)):

    print(f" fold {fold+1}/{3}, train data : {cfg.train_fragment[trn_idx]}, valid data : {cfg.train_fragment[val_idx]}")

    train_dataset = CustomTrainDataset(cfg.train_fragment[trn_idx], get_train_transforms())
    train_loader = DataLoader(train_dataset, batch_size=cfg.train_batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers, drop_last=False, collate_fn=collate_fn)

    valid_dataset = CustomValidDataset(cfg.train_fragment[val_idx], get_valid_transforms())
    valid_loader = DataLoader(valid_dataset, batch_size=cfg.valid_batch_size, shuffle=False, pin_memory=True, num_workers=cfg.num_workers, drop_last=False, collate_fn=collate_fn)


    scaler = amp.GradScaler()
    # criterion = DiceBCELoss()
    # criterion = FocalTverskyLoss(alpha=0.7, beta=0.3, gamma=1, smooth=1)
    # criterion = CompoundLoss()
    criterion = nn.BCEWithLogitsLoss()

    model = Net()
    model.to(cfg.device, non_blocking=True)

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(
            nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]


    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=cfg.lr, weight_decay=cfg.wd)

    total_samples = len(train_loader)
    num_warmup_steps = int((total_samples * cfg.num_epochs * 0.05) / cfg.accumulate_iter)
    num_training_steps = total_samples * cfg.num_epochs
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=int(num_training_steps), num_cycles=0.5)


    valid_dic = read_dic(cfg.train_fragment[val_idx][0])
    # del valid_dic['mask']
    # gc.collect()
    H,W = valid_dic['label'].shape
    pad0 = (cfg.crop_size - H % cfg.crop_stride)
    pad1 = (cfg.crop_size - W % cfg.crop_stride)
    valid_label = np.pad(valid_dic['label'], [(0, pad0), (0, pad1)], constant_values=0)
    valid_mask = np.pad(valid_dic['mask'], [(0, pad0), (0, pad1)], constant_values=0)
    H,W = valid_label.shape
    gt_shape = valid_label.shape

    # train and validation
    del train_dataset, valid_dataset
    gc.collect()

    best_valid_loss = 9999
    best_fbeta = 0
    early_stopping = EarlyStopping(patience=cfg.patience, verbose=1)
    for epoch in range(1, CFG.num_epochs+1):
        model.train()
        lrs = lr_scheduler.get_last_lr()[0]
        cur_lr = f"LR : {lrs:.7f}"
        pbar_train = enumerate(train_loader)
        pbar_train = tqdm(pbar_train, total=len(train_loader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}")
        mloss_train, mloss_val, val_metric = 0.0, 0.0, 0.0

        for i, batch in pbar_train:
            for k, v in batch.items():
                batch[k] = v.to(cfg.device, non_blocking=True)
            label = batch['labels'].unsqueeze(1)
            with amp.autocast(enabled=True):
                output = model(batch)
                loss = criterion(output['logit'], label)
            loss = loss / cfg.accumulate_iter
            scaler.scale(loss).backward()

            if ((i + 1) % cfg.accumulate_iter == 0) or (i + 1 == len(train_loader)):
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)

                if cfg.batch_scheduler:
                    lr_scheduler.step()

            mloss_train += loss.item() * cfg.accumulate_iter

            gpu_mem = f"Mem : {torch.cuda.memory_reserved() / 1E9:.3g}GB"
            pbar_train.set_description(("%10s  " * 3 + "%10s") % (f"Epoch {epoch}/{CFG.num_epochs}", gpu_mem, cur_lr,
                                                                  f"Loss: {mloss_train / (i + 1):.4f}"))


        model.eval()
        pbar_val = enumerate(valid_loader)
        pbar_val = tqdm(pbar_val, total=len(valid_loader), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}")


        infer_mask = make_infer_mask()
        final_pred_mask = torch.zeros(gt_shape, dtype=torch.float32, device='cuda')
        count = torch.zeros(gt_shape, dtype=torch.float32, device='cuda')

        for i, batch in pbar_val:
            for k, v in batch.items():
                batch[k] = v.to(cfg.device, non_blocking=True)
            label = batch['labels'].unsqueeze(1)
            xys = batch['xys']

            with torch.no_grad():
                with torch.cuda.amp.autocast(enabled=True):
                    output = model(batch)
                    pred_masks = output['ink']
                    loss = criterion(output['logit'], label)
                    mloss_val += loss.item()

            for j, xy in enumerate(xys):
                final_pred_mask[xy[1]:xy[3], xy[0]:xy[2]] += pred_masks[j, 0] * infer_mask
                count[xy[1]:xy[3], xy[0]:xy[2]] += infer_mask

            pbar_val.set_description(("%10s") % (f"Val Loss: {mloss_val / (i+1):.4f}"))

        final_pred_mask = final_pred_mask/(count+0.000001)
        final_pred_mask = final_pred_mask[:H,:W]
        final_pred_mask = final_pred_mask*torch.from_numpy(valid_mask).float().cuda()

        best_th = 0
        temp_best_fbeta = 0

        for threshold in np.arange(0.2, 0.75, 0.05):
            temp_fbeta = fbeta_score(final_pred_mask, torch.from_numpy(valid_label), threshold)

            if np.round(threshold,8) == 0.5:
                fbeta = temp_fbeta
                if fbeta>temp_best_fbeta:
                    temp_best_fbeta = fbeta
                    best_th = threshold
            print(f"Threshold : {threshold:.2f}\tFBeta : {temp_fbeta:.6f}")
        if best_fbeta < temp_best_fbeta:
            print(f"best_th : {best_th:.2f}\tFBeta : {temp_best_fbeta:.6f}")

            pred_mask = final_pred_mask.detach().cpu().numpy()
            plt.figure(figsize=(20, 16))
            plt.subplot(1,3,1)
            plt.title(f"image")
            plt.imshow(pred_mask)

            plt.subplot(1,3,2)
            plt.title(f"pred_ink")
            plt.imshow(np.where(pred_mask>0.5,1,0))

            plt.subplot(1,3,3)
            plt.title(f"labels")
            plt.imshow(valid_dic['label'])
            plt.show()

            print(metric_to_text(pred_mask, valid_label, valid_mask))

            print(f"{g_}Validation Score Increased from {best_fbeta} to {temp_best_fbeta}{sr_}")
            print('best valid loss achieved')
            best_valid_loss = mloss_val
            if not os.path.exists(cfg.colab_dir + f'/model/{cfg.model_name}'):
                os.mkdir(cfg.colab_dir + f'/model/{cfg.model_name}')
            torch.save(model.state_dict(),f"{cfg.colab_dir}/model/{cfg.model_name}/fold_{fold}_{cfg.model_name}_{cfg.date}.pt")

            best_fbeta = temp_best_fbeta

    #     if (epoch%10) == 0:
    #         print(metric_to_text(final_pred_mask.detach().cpu().numpy(), valid_dic["label"], valid_dic['mask']))

        gc.collect()
        torch.cuda.empty_cache()

        if early_stopping.score_validate(best_fbeta):
            break

        print()
    del train_loader, valid_loader, valid_dic
    gc.collect()


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# H,W = valid_dic['label'].shape
# pad0 = (cfg.crop_size - H % cfg.crop_stride)
# pad1 = (cfg.crop_size - W % cfg.crop_stride)
# valid_label = np.pad(valid_dic['label'], [(0, pad0), (0, pad1)], constant_values=0)
# gt_shape = valid_label.shape

In [ ]:
# H, W = valid_dic['label'].shape

# infer_mask = make_infer_mask()
# final_pred_mask = torch.zeros(gt_shape, dtype=torch.float32, device='cuda')
# count = torch.zeros(gt_shape, dtype=torch.float32, device='cuda')

# for i, batch in pbar_val:
#     for k, v in batch.items():
#         batch[k] = v.to(cfg.device, non_blocking=True)
# #         xys = batch['xys']
#     label = batch['labels'].unsqueeze(1)
# #         mask = batch['masks']
#     xys = batch['xys']

#     with torch.no_grad():
#         with torch.cuda.amp.autocast(enabled=True):
#             output = model(batch)
#             pred_masks = output['ink']
#             loss = criterion(output['logit'], label)
#             mloss_val += loss.item()
# #             mloss_val += criterion(pred_masks, label.unsqueeze(1)).item()
# #             pred_masks = torch.sigmoid(pred_masks)

#     for j, xy in enumerate(xys):
#         final_pred_mask[xy[1]:xy[3], xy[0]:xy[2]] += pred_masks[j, 0] * infer_mask
#         # count[xy[1]:xy[3], xy[0]:xy[2]] += infer_mask

#     pbar_val.set_description(("%10s") % (f"Val Loss: {mloss_val / (i+1):.4f}"))

# # final_pred_mask = final_pred_mask/(count+0.000001)
# final_pred_mask = final_pred_mask[:H,:W]
# final_pred_mask = final_pred_mask*torch.from_numpy(valid_dic['mask']).float().cuda()

In [ ]:
# for threshold in np.arange(0.2, 0.75, 0.05):
#     temp_fbeta = fbeta_score(final_pred_mask, torch.from_numpy(valid_dic['label']), threshold)
#     print(temp_fbeta)

In [ ]:
# pred_mask = final_pred_mask.detach().cpu().numpy()
# plt.figure(figsize=(20, 16))
# plt.subplot(1,3,1)
# plt.title(f"image")
# plt.imshow(pred_mask)

# plt.subplot(1,3,2)
# plt.title(f"pred_ink")
# plt.imshow(np.where(pred_mask>0.5,1,0))

# plt.subplot(1,3,3)
# plt.title(f"labels")
# plt.imshow(valid_dic['label'])
# plt.show()

In [ ]:
# fold1 - cv.0.639 valid 2a
# fold2 - cv 0.651 valid 2b
# fold3 - cv 0.653 valid 2c
# fold4 - cv 0.624 valid 3
# fold5 - cv 0.646   valid 1

In [ ]:
# fold1 -cv  0.634
# 16 - 23 1
# 17 - 24 2
# 18 - 25 3
# 19 - 26 4 - 높아짐
# 20 - 27 5
# 21 - 28 6
# 22 - 29 7
# 23 - 30 8
# 24 - 31 9
# 25 - 32 10
# 26 - 33 11


In [ ]:

# def do_random_cutmix(
#     self,
#     image,
#     label,
#     mask,
#     index,
#     valid_mask_threshold=0.8,
#     valid_label_threshold=0.1,
#     mode = 'mask'
#     ):
#     w, h = image.shape[:2]
#     ws = w // 2
#     hs = h // 2

#     xc, yc = [int(random.uniform(w*0.35, h*0.65)) for _ in range(2)]
#     indexes = [index] + [random.randint(0, index) for _ in range(3)]


#     result_img = np.full((imsize, imsize, 3), 1, dtype=np.float32)

#     for i, index in enumerate(indexes):
#         image = load_image(path, index)

#         #top left
#         if i == 0:
#             x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
#             x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
#         elif i == 1:  # top right
#             x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
#             x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
#         elif i == 2:  # bottom left
#             x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
#             x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
#         elif i == 3:  # bottom right
#             x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
#             x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)

#         result_img[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]

#     return result_img